# Summary

Crawl a bunch of sites and save data to BQ

In [2]:
import sys, os

import pandas as pd
import numpy as np

# Asynchronous work
import asyncio

from google.oauth2 import service_account
import pandas_gbq

# Web crawling and scraping tools class
sys.path.insert(0, "../../scrapers/")
import web_scraper as ws
import web_crawler as wc


## Test the site scraper on one site

In [2]:
turl = "https://en.wikipedia.org/wiki/Chaffey_College"
turl = "https://asdjhfbads.jhbf"
turl = "https://www.cccco.edu/"

test = await ws.webScraper.visit_page(url=turl)

test.crawl_results.keys()
# type(test.crawl_results)


[INFO] Starting Chromium download.
100%|██████████| 141M/141M [00:05<00:00, 25.3Mb/s] 
[INFO] Beginning extraction
[INFO] Chromium extracted to: /Users/stephengodfrey/Library/Application Support/pyppeteer/local-chromium/1181205


dict_keys(['url', 'html_code_string', 'soup', 'ptag_text', 'atag_urls'])

## Web Crawler

## Set up GCP credentials

In [ ]:
# Set GCP credentials
creds_path = "../../../data/environment/"
creds_file = "eternal-bongo-435614-b9-bf6a5e630e44.json"
credentials = service_account.Credentials.from_service_account_file(filename=os.path.join(creds_path,
                                                                                          creds_file))

# Set credentials
pandas_gbq.context.credentials = credentials


## Get a list of previously crawled websites

In [ ]:
# Query the BigQuery crawl results table to get a list of sites already crawled
# Identify the table; note the database.table naming schema
table_id = "ccc_polasst.crawl_data"
project_id = "eternal-bongo-435614-b9"

# Create a sql statement
sql = ("SELECT DISTINCT url FROM `{}` ").format(table_id)
sql = ("SELECT * FROM `{}` ").format(table_id)

# Use Pandas-gbq to read from the BQ table
# df_pc = pandas_gbq.read_gbq(sql, project_id=project_id)

########### Get a list of previously crawled sites
########### This needs some work - don't crawl may not work since we still need the URLs
# crawled_urls = df_pc["url"].unique().tolist()

# dont_crawl_urls = []

# len(crawled_urls)

## Crawl websites (and write results to OneDrive)

In [3]:
# Crawl seed_url and web links found on it and its child pages
# seed_url = "https://www.cccco.edu"
seed_url = "https://en.wikipedia.org/wiki/California_Community_Colleges"
dont_crawl_urls = []

crawler = wc.webCrawler(seed_url=seed_url)
crlres = await crawler.crawl_sites(dont_crawl_urls=dont_crawl_urls,
                                   depth=10,
                                   width=300)


 10%|█         | 1/10 [00:04<00:37,  4.18s/it]

Depth level finished: 1: 1 URLs crawled; 552 URLs in to_crawl_urls; 2 URLs in dont_crawl_urls
Batch 1 saved to disk
Batch 2 saved to disk


 20%|██        | 2/10 [11:04<52:02, 390.27s/it]

Depth level finished: 2: 301 URLs crawled; 39579 URLs in to_crawl_urls; 602 URLs in dont_crawl_urls
Batch 3 saved to disk
Batch 4 saved to disk
Batch 5 saved to disk


 30%|███       | 3/10 [22:03<59:49, 512.73s/it]

...

Depth level finished: 8: 1003 URLs crawled; 500 URLs in to_crawl_urls; 3710 URLs in dont_crawl_urls


 90%|█████████ | 9/10 [36:47<01:34, 94.24s/it] 

Batch 10 saved to disk
Depth level finished: 9: 1004 URLs crawled; 952 URLs in to_crawl_urls; 4310 URLs in dont_crawl_urls


100%|██████████| 10/10 [36:49<00:00, 220.96s/it]

Depth level finished: 10: 1005 URLs crawled; 0 URLs in to_crawl_urls; 4910 URLs in dont_crawl_urls
Batch 11 saved to disk


In [ ]:
dir(crlres)
# print("This crawl dataframe is {} rows.".format(len(df)))


## Read local crawl results

In [5]:
crawl_data_path  = ("/Users/stephengodfrey/OneDrive - numanticsolutions.com"
                    "/Engagements/Projects/ccc_policy_assistant/data/crawls")
crawl_file = "enwikipediaorg_2024Dec11_10.csv"

os.listdir(crawl_data_path)

dft = pd.read_csv(filepath_or_buffer=os.path.join(crawl_data_path,crawl_file))


## Save results to BigQuery

In [ ]:
# Identify the table; note the database.table naming schema
table_id = "ccc_polasst.crawl_data"
table_id = "ccc_polasst.crawl_data_tes"
project_id = "eternal-bongo-435614-b9"

# Use Pandas-gbq to write to the BQ table
if_exists = "replace"
# if_exists = "append"

# pandas_gbq.to_gbq(dataframe=df, destination_table=table_id,
#                   project_id=project_id, if_exists=if_exists)

